<a href="https://colab.research.google.com/github/MorganChidley/Final-Year-Project/blob/main/ml_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load the dataset
file_path = "MyDataSET.csv"
data = pd.read_csv(file_path)

# Display the first few rows of the data
data.head()

,URL,ClassLabel
0,https://keraekken-loagginnusa.godaddysites.com/,0
1,https://metamsk01lgiix.godaddysites.com/,0
2,http://myglobaltech.in/,0
3,http://djtool-for-spotify.com/,0
4,https://scearmcoommunnlty.com/invent/freind/get,0


In [ ]:
import pandas as pd

# Load the dataset
file_path = "MyDataSET.csv"
data= pd.read_csv(file_path)

# Get the shape of the data (number of rows and columns)
data.shape

(905, 2)

In [ ]:
import pandas as pd

# Load the dataset
file_path = "MyDataSET.csv"
data= pd.read_csv(file_path)

# Check for missing values
data.isnull().sum()

,0
URL,0
ClassLabel,0


In [ ]:
import pandas as pd

# Load the dataset
file_path = "MyDataSET.csv"
data= pd.read_csv(file_path)

# Remove duplicate rows
data.drop_duplicates(inplace=True)

# Check the new shape of the dataset
data.shape



(903, 2)

In [ ]:
import pandas as pd
import re
from urllib.parse import urlparse

# Feature extraction functions

def url_length(url):
    """Returns the length of the URL."""
    return len(url)

def has_ip_address(url):
    """Checks if the URL contains an IP address."""
    ip_pattern = r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}"
    return 1 if re.search(ip_pattern, url) else 0

def count_special_chars(url):
    """Counts occurrences of special characters."""
    special_chars = ["-", "_", "%", "/", ".", "#"]
    return {char: url.count(char) for char in special_chars}

def count_subdomains(url):
    """Counts the number of subdomains."""
    parsed_url = urlparse(url).netloc
    return parsed_url.count('.')

def has_https(url):
    """Checks if the URL uses HTTPS."""
    return 1 if urlparse(url).scheme == "https" else 0


def count_query_parameters(url):
    """Counts the number of query parameters in the URL."""
    query = urlparse(url).query
    return len(query.split("&")) if query else 0

# Apply feature extraction

data["url_length"] = data["URL"].apply(url_length)
data["subdomain_count"] = data["URL"].apply(count_subdomains)
data["https"] = data["URL"].apply(has_https)
data["has_ip_address"] = data["URL"].apply(has_ip_address)
data["query_parameters_count"] = data["URL"].apply(count_query_parameters)

# Extract special character counts only once
if not all(char in data.columns for char in ["-", "_", "%", "/", ".", "#"]):
    special_chars_df = data["URL"].apply(count_special_chars).apply(pd.Series)
    data = pd.concat([data, special_chars_df], axis=1)

# Remove duplicate columns
data = data.loc[:, ~data.columns.duplicated()]

# Converts all feature changes to a new csv file
data.to_csv("modified_dataset.csv", index=False)

# Display first few rows with new features
data.head()

,URL,ClassLabel,url_length,subdomain_count,https,has_ip_address,query_parameters_count,-,_,%,/,.,#
0,https://keraekken-loagginnusa.godaddysites.com/,0,47,2,1,0,0,1,0,0,3,2,0
1,https://metamsk01lgiix.godaddysites.com/,0,40,2,1,0,0,0,0,0,3,2,0
2,http://myglobaltech.in/,0,23,1,0,0,0,0,0,0,3,1,0
3,http://djtool-for-spotify.com/,0,30,1,0,0,0,2,0,0,3,1,0
4,https://scearmcoommunnlty.com/invent/freind/get,0,47,1,1,0,0,0,0,0,5,1,0


In [ ]:
from pickle import TRUE
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from IPython.display import display

# Load Dataset
data = pd.read_csv("modified_dataset.csv")

# Feature Selection
features = ['url_length', 'subdomain_count', 'https', "-", "_", "%", "/", ".", "#",
            "has_ip_address", "query_parameters_count"]
X = data[features]
y = data['ClassLabel']  # Target variable

# Handle missing values (fill with median)
X.fillna(X.median(), inplace=True)

# Feature Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train and evaluate multiple models
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "SVM": SVC(probability=True),
    "MLP": MLPClassifier(hidden_layer_sizes=(100,), max_iter=500)
}

# Cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Evaluate models using cross-validation
kfold_results = {}
for model_name, model in models.items():
    scores = cross_val_score(model, X_scaled, y, cv=kf, scoring='accuracy')
    kfold_results[model_name] = {
        "Model": model_name,
        "Mean Accuracy": np.mean(scores),
        "Standard Deviation": np.std(scores)
    }

# Store results in a list of dictionaries
results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
# Display the DataFrame

   # Extract precision, recall, and F1-score for each class
    report_dict = classification_report(y_test, y_pred, output_dict=True)
    precision = report_dict['weighted avg']['precision']
    recall = report_dict['weighted avg']['recall']
    f1_score = report_dict['weighted avg']['f1-score']

    results.append({
        'Model': name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1_score,
        'ROC-AUC': roc_auc,
        'Confusion Matrix': conf_matrix,
    })

# Create a Pandas DataFrame from the results
results_df = pd.DataFrame(results)
kfold_results_df = pd.DataFrame(kfold_results.values())

# Reorder columns
kfold_results_df = kfold_results_df[['Model', 'Mean Accuracy', 'Standard Deviation']]

# Display the DataFrame
display(results_df)
display(kfold_results_df)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Load Dataset
data = pd.read_csv("modified_dataset.csv")

# Drop the 'ClassLabel' column
data = data.drop('ClassLabel', axis=1)

data.hist(figsize=(15, 30))  # Adjust figsize as needed
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Load Dataset
data = pd.read_csv("modified_dataset.csv")

# Drop the 'ClassLabel' column
data = data.drop('ClassLabel', axis=1)

# Select numeric columns
numeric_data = data.select_dtypes(include=np.number)

# Calculate correlation matrix
correlation_matrix = numeric_data.corr()

# Display correlation matrix as a styled table
correlation_table = correlation_matrix.style.background_gradient(cmap='coolwarm')

correlation_table

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import mutual_info_classif
from IPython.display import display # Import for display function

# Load dataset
data = pd.read_csv("modified_dataset.csv")

X = data.drop(['ClassLabel', "URL"], axis=1)
y = data['ClassLabel']

# Mutual Information
mutual_info = mutual_info_classif(X, y)
feature_scores = pd.DataFrame({'Feature': X.columns, 'Mutual_Information': mutual_info})
feature_scores = feature_scores.sort_values(by=['Mutual_Information'], ascending=False)


# Display mutual information scores in a table
display(feature_scores)

